<a href="https://colab.research.google.com/github/caropilardiaz/Mentoria-Imagenes_Drones/blob/master/2_0_Georeferencia_AyC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Ingeniería de Features
##Diplodatos 2020, Mentorias
##Mentora:Carolina Del Pilar Díaz

###Instalación de librerías

In [ ]:
!pip install geopandas
!apt install libspatialindex-dev
!pip install rtree
!pip install osmnx
!pip install contextily

### Cargar informacion

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from IPython.display import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString
import contextily as ctx

In [ ]:
# Editar con el path correspondiente a sus datos
data_path = './drive/My Drive/Mentoria_Drones_master'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DATA_DIRECTORY = '/content/drive/My Drive/Mentoria-Drones-master/data/'

### Cargamos los datos


In [ ]:
import os

In [ ]:
dataframe = pd.read_csv(os.path.join(DATA_DIRECTORY,'Dron1.csv'))


## Manejo de objetos graficos con Shapely

https://shapely.readthedocs.io/en/latest/manual.html

##Leemos los datos dentro de un dataframe de pandas

In [ ]:
dron = pd.read_csv(os.path.join(DATA_DIRECTORY,'Dron1.csv'), thousands='.') # Indicamos el separador de miles

In [ ]:
dron.head()

In [ ]:
dron.dtypes

###Podemos convertirlo en un GeoDataframe indicando las columnas que contienen la información geografica

In [ ]:
dron_gp = gpd.GeoDataFrame(dron, geometry=gpd.points_from_xy(dron.Longitude, dron.Latitude))

In [ ]:
dron_gp.shape, dron_gp.columns

###Objetos geométricos básicos

In [ ]:
dron_gp.geometry.head()

In [ ]:
p = Point(1,0)
o = Point(0,0)

In [ ]:
p.length, p.area, p.bounds, p.x, p.y

In [ ]:
l = LineString([(1,1),(-1,-1),(-1,-0.5)])
l

In [ ]:
l.area, l.length, l.bounds, l.xy, l.centroid.xy

In [ ]:
c = Polygon([(0,0),(0,1),(1,1),(1,0),(0.5,-0.5)])
c

In [ ]:
c.area, c.length, c.bounds, c.centroid.x, c.centroid.y

###Operaciones Basicas

In [ ]:
p.distance(l) # Distancia del punto a linea

In [ ]:
l.contains(p) # Contiene

In [ ]:
l.intersects(c) # Interseccion

###Gráficamos diseño experimental

In [ ]:
# Graficamos y ajustamos el tamaño
dron_gp.plot(figsize=(10,10))

In [ ]:
# Podemos usar cualquier columna para codificar los colores del grafico, con el parametro column
dron_gp.plot(figsize=(10,10), column='Variedad')

In [ ]:
# Incluso utilizando variables numericas
dron_gp.plot(figsize=(10,10), column='Peso de 1000 granos', legend=True)

In [ ]:
# Incluso utilizando variables numericas
dron_gp.plot(figsize=(10,10), column='RDTO', legend=True)

In [ ]:
# Incluso utilizando variables numéricas
dron_gp.plot(figsize=(10,10), column='ZADOCK', legend=True)

##Joints Espaciales

### Sistemas de Coordenadas de Referencia, CRS
https://geopandas.org/projections.html

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
world.crs

In [ ]:
ax = world.plot()
ax.set_title('WGS 84 - EPSG 4326')

In [ ]:
ax = world.to_crs('EPSG:3395').plot()
ax.set_title('Mercator - EPSG 3395')

###Conversion a un sistema de referencia UTM local métrico
Primero encontramos que nuestra zona correspondiente es 20J o (20L)

https://www.latlong.net/lat-long-utm.html

Luego encontramos que el sistema correspondiente es EPSG: 4326 WGS 84 


https://epsg.io/4326


In [ ]:
dron_gp = gpd.GeoDataFrame(dron, geometry=gpd.points_from_xy(dron.Longitude, dron.Latitude))

In [ ]:
dron_gp.crs

In [ ]:
dron_gp.geometry

## Generacion de features a partir de variables georeferenciadas

####Distancia Euclideana a puntos de referencia dentro del dataset
####Ejemplos:

#####Distancia a los mayores rendimientos de cada parcela
#####Distancia a pivot de riego
#####Distancia a ruta más cercanas

In [ ]:
dist_dron_gp = dron_gp.distance(dron_gp.geometry[1])
dist_dron_gp.head()

In [ ]:
dist_dron_gp.hist()

##Generacion de features a traves de árboles de busqueda espaciales

##Calculo de densidades

In [ ]:
from scipy.spatial import cKDTree

In [ ]:
# Convertimos las coordenas a una matriz
dron_loc_array = [x.coords[0] for x in dron_gp.geometry]

###Pixeles en un radio de 1 metro

In [ ]:
radio =30
tree = cKDTree(dron_loc_array)
near_points = tree.query_ball_point(dron_loc_array, radio)
n_near_points = [len(p) for p in near_points]

###Podemos convertir estos datos en variables de diferentes formas

###Distancia al elemento mas cercano

In [ ]:
med_tree = cKDTree(dron_loc_array)
d, i = med_tree.query(dron_loc_array, k=2, n_jobs=-1)

In [ ]:
sns.distplot(n_near_points)

##Generacion de Features a partir de Grafos
https://networkx.github.io/documentation/stable/tutorial.html

Hasta el momento hemos utilizado la informacion sobre ubicacion de nuestros items pero no la interconexion entre los mismos.

Para captar esta relacion queremos expresar nuestro diseño como un grafo.